# Cython essentials

Cython is:
*  A programming language blend of Python and the type system of C/C++.
*  A compiler for Cython -> C\C++.

It feels like Python while providing easy access to C.

## Comparint Python, C and Cython
Starting with Python...

In [1]:
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a+b, a
    return a


In [2]:
[fib(i) for i in range(1,11)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Then with C.

In [3]:
code = """
#include <stdio.h>

int cfib(int n);


int cfib(int n){
  int a=0, b=1, tmp;
  for(int i=0; i<n; ++i){
    tmp = a; a = a + b; b = tmp;
  }
  return a;
}


void main(){
  for(int i=1; i<=10; i++){
    printf("%d,", cfib(i));
  }
}
"""

with open("cfib.c", "w") as target_file:
    target_file.write(code)


In [4]:
! gcc cfib.c

In [5]:
! a

1,1,2,3,5,8,13,21,34,55,


Download https://sourceforge.net/projects/mingw-w64/

Choose x86_64 in architecture

In [6]:
! C:\"Program Files"\mingw-w64\x86_64-8.1.0-posix-seh-rt_v6-rev0\mingw64\bin\gcc -m64 -fpic -c cfib.c -o cfib.o

In [7]:
! C:\"Program Files"\mingw-w64\x86_64-8.1.0-posix-seh-rt_v6-rev0\mingw64\bin\gcc -shared -o libcfib.so cfib.o

In [8]:
import ctypes

libcfib = ctypes.cdll.LoadLibrary("libcfib.so")

In [9]:
cfib = libcfib.cfib

[cfib(i) for i in range(1,11)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Finally, running with Cython...

In [10]:
%load_ext Cython

In [11]:
%%cython

def cyfib(int n):
    cdef int a = 0
    cdef int b = 1
    for i in range(n):
        a, b = a+b, a
    return a


In [12]:
[cyfib(i) for i in range(1,11)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

So we have this fib function implemented in Python (fib), C (cfib) and Cython (cyfib)...

In [13]:
%%timeit
[fib(i) for i in range(1,101)]

266 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%%timeit
[cfib(i) for i in range(1,101)]

43.5 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%%timeit
[cyfib(i) for i in range(1,101)]

8.77 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## The basic build of hello world...
See the basic [cython tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) for hello world.

In [36]:
code = """
print("Hello World")
"""

with open("helloworld.pyx", "w") as target_file:
    target_file.write(code)

    
code = """
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("helloworld.pyx")
)
"""

with open("setup.py", "w") as target_file:
    target_file.write(code)


In [37]:
%%bash
cat helloworld.pyx


print("Hello World")


In [38]:
%%bash
cat setup.py


from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("helloworld.pyx")
)


In [40]:
%%bash
source activate py36
python setup.py build_ext --inplace

running build_ext
building 'helloworld' extension
creating build
creating build\temp.win-amd64-3.6
creating build\temp.win-amd64-3.6\Release
C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.20.27508\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Anaconda2\envs\py36\include -IC:\Anaconda2\envs\py36\include "-IC:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.20.27508\include" "-IC:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\include\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\shared" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\winrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\cppwinrt" /Tchelloworld.c /Fobuild\temp.win-amd64-3.6\Release\helloworld.obj
helloworld.c
C:\Program Files (x86)\Microsoft Visual Studio\2

I've run this on Windows, so I have 'helloworld.cp36-win_amd64.pyd' in the current directory.

In [41]:
import os

os.path.isfile("helloworld.cp36-win_amd64.pyd")

True

In [43]:
import helloworld

Hello World


## Wrapping C code with Cython

In [16]:
code = """
int cfib(int n);
"""

with open("cfib.h", "w") as target_file:
    target_file.write(code)


In [17]:
%%bash
cat cfib.c


#include <stdio.h>

int cfib(int n);


int cfib(int n){
  int a=0, b=1, tmp;
  for(int i=0; i<n; ++i){
    tmp = a; a = a + b; b = tmp;
  }
  return a;
}


void main(){
  for(int i=1; i<=10; i++){
    printf("%d,", cfib(i));
  }
}


In [18]:
%%bash
cat cfib.h


int cfib(int n);


Note that the %%cython cell magic [doesn't help](https://stackoverflow.com/questions/37426534/how-can-i-import-an-external-c-function-into-an-ipython-notebook-using-cython) in this case...

We'll have to put it in a Python file.

In [49]:
code = """

cdef extern from "cfib.h":
    int cfib(int n)


def fib(n):
    return cfib(n)

"""

with open("cfib.pyx", "w") as target_file:
    target_file.write(code)

In [50]:
code = """
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("cfib.pyx")
)
"""

with open("setup.py", "w") as target_file:
    target_file.write(code)

In [ ]:
%%bash
source activate py36
python setup.py build_ext --inplace

***